In [ ]:
import { Ollama } from '@langchain/community/llms/ollama';
import { HumanMessage } from "@langchain/core/messages";
import { StringOutputParser } from "@langchain/core/output_parsers";

const llm = new Ollama({ 
  baseUrl: 'http://localhost:11434',
  model: 'llama3.1:8b',
});

const parser = new StringOutputParser();
const chain = llm.pipe(parser);

await chain.invoke([
  new HumanMessage('Tell me a joke')
]);

In [ ]:
// StructedOutputParser
import { StructuredOutputParser } from "langchain/output_parsers";
import { PromptTemplate } from "@langchain/core/prompts";
import { Ollama } from '@langchain/community/llms/ollama';

const llm = new Ollama({ 
  baseUrl: 'http://localhost:11434',
  model: 'llama3.1:8b',
});

const parser = StructuredOutputParser.fromNamesAndDescriptions({
  answer: "用户问题的答案",
  evidence: "你回答用户问题所依据的答案",
  confidence: "问题答案的可信度评分，格式是百分数",
});

// console.log(parser.getFormatInstructions());

const prompt = PromptTemplate.fromTemplate('尽可能的回答用的问题 \n{instructions} \n{question}');
const chain = prompt.pipe(llm).pipe(parser);
const res = await chain.invoke({
  question: "蒙娜丽莎的作者是谁？是什么时候绘制的",
  instructions: parser.getFormatInstructions()
});
console.log(res);

In [ ]:
// ListOutputParser
import { CommaSeparatedListOutputParser } from "@langchain/core/output_parsers";
import { PromptTemplate } from "@langchain/core/prompts";
import { Ollama } from '@langchain/community/llms/ollama';

const llm = new Ollama({ 
  baseUrl: 'http://localhost:11434',
  model: 'llama3.1:8b',
});
const parser = new CommaSeparatedListOutputParser();
const prompt = PromptTemplate.fromTemplate("列出3个 {country} 的著名的互联网公司.\n{instructions}")
console.log(parser.getFormatInstructions());

const chain = prompt.pipe(llm).pipe(parser);
const res = await chain.invoke({
  country: 'America',
  instructions: parser.getFormatInstructions(),
});

console.log(res);

In [ ]:
/*
 * @Author: NanluQingshi
 * @Date: 2026-02-04 23:15:43
 * @LastEditors: NanluQingshi
 * @LastEditTime: 2026-02-05 00:35:01
 * @Description: OutputFixParser
 */
import { z } from 'zod';
import { StructuredOutputParser, OutputFixingParser } from "langchain/output_parsers";
import { PromptTemplate } from "@langchain/core/prompts";
import { Ollama } from '@langchain/community/llms/ollama';

const llm = new Ollama({ 
  baseUrl: 'http://localhost:11434',
  model: 'llama3.1:8b',
});

const schema = z.object({
  answer: z.string().describe('用户问题的答案'),
  confidence: z.number().min(0).max(100).describe('问题答案的可信度评分，满分 100'),
});
const parser = StructuredOutputParser.fromZodSchema(schema);
const prompt = PromptTemplate.fromTemplate("尽可能的回答用的问题 \n{instructions} \n{question}");

// const wrongOutput = {
//   "answer": "蒙娜丽莎的作者是达芬奇，大约在16世纪初期（1503年至1506年之间）开始绘制。",
//   "sources": "90%" 
// };

// const fixParser = OutputFixingParser.fromLLM(llm, parser);
// const output = await fixParser.parse(JSON.stringify(wrongOutput));
// console.log(output);

const chain = prompt.pipe(llm).pipe(parser);
const res = await chain.invoke({
    question: "蒙娜丽莎的作者是谁？是什么时候绘制的",
    instructions: parser.getFormatInstructions(),
});
                               
console.log(res);